<a href="https://colab.research.google.com/github/SarvAster/Graphs-GNNs-KGs/blob/main/Node_Prop_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import os
print("PyTorch has version {}".format(torch.__version__))
torch_version = str(torch.__version__)
scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
!pip install torch-scatter -f $scatter_src
!pip install torch-sparse -f $sparse_src
!pip install torch-geometric
!pip install ogb

from torch_geometric.datasets import TUDataset

root = './enzymes'
name = 'ENZYMES'

# The ENZYMES dataset
pyg_dataset= TUDataset(root, name)

# You will find that there are 600 graphs in this dataset
print(pyg_dataset)


PyTorch has version 2.5.1+cu121
Looking in links: https://pytorch-geometric.com/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 25.2 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.5 MB/s eta 0:00:00


Processing...


ENZYMES(600)


Done!


In [2]:
def get_num_classes(pyg_dataset):
  return pyg_dataset.num_classes

def get_num_features(pyg_dataset):
  num_features = pyg_dataset.num_features
  return num_features

num_classes = get_num_classes(pyg_dataset)
num_features = get_num_features(pyg_dataset)
print("{} dataset has {} classes".format(name, num_classes))
print("{} dataset has {} features".format(name, num_features))

ENZYMES dataset has 6 classes
ENZYMES dataset has 3 features


In [3]:
def get_graph_class(pyg_dataset, idx):
  label = pyg_dataset[idx].y.item()
  return label

# Here pyg_dataset is a dataset for graph classification

graph_0 = pyg_dataset[0]
print(graph_0)
print(pyg_dataset[0].y.item())
idx = 100
label = get_graph_class(pyg_dataset, idx)
print('Graph with index {} has label {}'.format(idx, label))

Data(edge_index=[2, 168], x=[37, 3], y=[1])
5
Graph with index 100 has label 4


In [4]:
def get_graph_num_edges(pyg_dataset, idx):
  data = pyg_dataset[idx]
  edges = set()
  for i in range(data.edge_index.shape[1]):
    u,v = (data.edge_index[0,i].item(), data.edge_index[1,i].item())
    edges.add((max(u,v), min(u,v)))
  return len(edges)

idx = 200
num_edges = get_graph_num_edges(pyg_dataset, idx)
print('Graph with index {} has {} edges'.format(idx, num_edges))
print(pyg_dataset[idx].num_edges)

Graph with index 200 has 53 edges
106


In [5]:
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset

dataset_name = 'ogbn-arxiv'
# Load the dataset and transform it to sparse tensor
dataset = PygNodePropPredDataset(name=dataset_name,
                                transform=T.ToSparseTensor())
print('The {} dataset has {} graph'.format(dataset_name, len(dataset)))

# Extract the graph
data = dataset[0]
print(data)

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:10<00:00,  7.88it/s]


Extracting dataset/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 2074.33it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 2565.32it/s]

Saving...



Done!
/usr/local/lib/python3.10/dist-packages/ogb/nodeproppred/dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.pro

The ogbn-arxiv dataset has 1 graph
Data(num_nodes=169343, x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=1166243])


In [6]:
def graph_num_features(data):
  return data.num_features

num_features = graph_num_features(data)
print('The graph has {} features'.format(num_features))

The graph has 128 features


In [7]:
import torch
import pandas as pd
import torch.nn.functional as F
print(torch.__version__)

# The PyG built-in GCNConv
from torch_geometric.nn import GCNConv

import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

2.5.1+cu121


In [8]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name,
                                transform=T.ToSparseTensor())
data = dataset[0]

# Make the adjacency matrix to symmetric
data.adj_t = data.adj_t.to_symmetric()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# If you use GPU, the device should be cuda
print('Device: {}'.format(device))

data = data.to(device)
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

/usr/local/lib/python3.10/dist-packages/ogb/nodeproppred/dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_

Device: cuda


In [14]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):

        super(GCN, self).__init__()
        self.num_layers = num_layers
        # A list of GCNConv layers
        self.convs = torch.nn.ModuleList()

        # A list of 1D batch normalization layers
        self.bns = torch.nn.ModuleList()
        self.convs.append(GCNConv(input_dim,hidden_dim))
        for _ in range(self.num_layers-1):
            self.convs.append(GCNConv(hidden_dim,hidden_dim))
            self.bns.append(torch.nn.BatchNorm1d(hidden_dim))

        self.convs.append(GCNConv(hidden_dim,output_dim))

        # The log softmax layer
        self.softmax = torch.nn.LogSoftmax()

        ## The parameters you can set for GCNConv include 'in_channels' and
        ## 'out_channels'. For more information please refer to the documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv
        ## The only parameter you need to set for BatchNorm1d is 'num_features'
        ## https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html)

        # Probability of an element getting zeroed
        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i in range(self.num_layers-1):
          x= self.convs[i](x,adj_t)
          x = self.bns[i](x)
          x = torch.nn.functional.relu(x)
          x = torch.nn.functional.dropout(x,self.dropout)

        x = self.convs[-1](x,adj_t)
        if self.return_embeds:
          return x
        else:
          return self.softmax(x)

In [15]:
def train(model, data, train_idx, optimizer, loss_fn):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.adj_t)
    loss = loss_fn(out[train_idx], data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

In [16]:
# Test function here
@torch.no_grad()
def test(model, data, split_idx, evaluator, save_model_results=False):
    # TODO: Implement a function that tests the model by
    # using the given split_idx and evaluator.
    model.eval()

    # The output of model on all data
    out = model(data.x, data.adj_t)

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    if save_model_results:
      print ("Saving Model Predictions")

      data = {}
      data['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

      df = pd.DataFrame(data=data)
      # Save locally as csv
      df.to_csv('ogbn-arxiv_node.csv', sep=',', index=False)


    return train_acc, valid_acc, test_acc

In [24]:
args = {
  'device': device,
  'num_layers': 5,
  'hidden_dim': 512,
  'dropout': 0.3,
  'lr': 0.005,
  'epochs': 100,
}
args
model = GCN(data.num_features, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbn-arxiv')

In [25]:
# Training should take <10min using GPU runtime
import copy
# reset the parameters to initial random value
model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = F.nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, optimizer, loss_fn)
  result = test(model, data, split_idx, evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch: 01, Loss: 4.3006, Train: 25.13%, Valid: 29.23% Test: 26.23%
Epoch: 02, Loss: 2.1855, Train: 20.34%, Valid: 23.82% Test: 30.57%
Epoch: 03, Loss: 2.8909, Train: 26.86%, Valid: 30.13% Test: 34.70%
Epoch: 04, Loss: 1.9346, Train: 22.85%, Valid: 22.84% Test: 27.61%
Epoch: 05, Loss: 1.6702, Train: 24.34%, Valid: 22.13% Test: 26.88%
Epoch: 06, Loss: 1.6081, Train: 30.24%, Valid: 25.30% Test: 29.38%
Epoch: 07, Loss: 1.5378, Train: 34.25%, Valid: 26.03% Test: 28.03%
Epoch: 08, Loss: 1.4375, Train: 32.06%, Valid: 20.66% Test: 18.76%
Epoch: 09, Loss: 1.3841, Train: 29.09%, Valid: 16.41% Test: 13.61%
Epoch: 10, Loss: 1.3577, Train: 27.32%, Valid: 15.16% Test: 12.33%
Epoch: 11, Loss: 1.3337, Train: 27.19%, Valid: 15.41% Test: 12.62%
Epoch: 12, Loss: 1.3063, Train: 29.03%, Valid: 16.56% Test: 13.60%
Epoch: 13, Loss: 1.2794, Train: 31.30%, Valid: 18.23% Test: 14.99%
Epoch: 14, Loss: 1.2488, Train: 32.77%, Valid: 19.59% Test: 16.75%
Epoch: 15, Loss: 1.2288, Train: 33.16%, Valid: 21.72% Test: 20

In [26]:
  best_result = test(best_model, data, split_idx, evaluator, save_model_results=True)
  train_acc, valid_acc, test_acc = best_result
  print(f'Best model: '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

Saving Model Predictions
Best model: Train: 74.05%, Valid: 71.60% Test: 70.75%
